# 通道套利策略：Bollinger Band (布林帶)

- **Bollinger Band(布林帶)**，假設股票的價格是服從**常態分佈**
- 從分配的中心點，也就是平均值，往**兩邊加減兩倍的標準差的範圍，占了所有範圍大約 95% 的比例**
- 短期的上漲或下跌，勢必會回復到常態
- 給予一段觀察期(通常是20天)，用前20日的資料算出平均值與標準差
- 若接下來的股價是超出平均值 + 兩個標準差，代表**不正常上漲**，所以就**放空股票，直到回到中心點平倉**
- 若接下來的股價是超出平均值 - 兩個標準差，代表**不正常下跌**，所以就**買入股票，直到回到中心點平倉**

由於平均值，與平均值 +/- 兩個標準差三個值畫成圖後，會像一個通道一樣，所以這操作又被稱爲**通道套利策略**
![](https://www.dropbox.com/s/fx4odaqpkxu9z1p/bollinger_band.PNG?dl=1)

In [2]:
import xlwings as xw
import time

In [3]:
# 打開你的 tsmc_back_test.xlsx 檔案
wb = xw.Book(r'C:\Users\user\Desktop\L5\bollinger_band.xlsm')
wb

<Book [bollinger_band.xlsm]>

In [4]:
tsmc_sheet = wb.sheets['2330']

In [5]:
# 截取出我們需要的股價資料
from xlwings.constants import Direction
# 偵測最後一筆資料
num_of_rows = tsmc_sheet.range('B1').end(Direction.xlDown).row

# xlwings 會把 excel一個範圍的值以一個清單表示
tsmc_prices = tsmc_sheet.range((2, "B"), (num_of_rows, "B")).value
tsmc_prices

[1455.22,
 1399.42,
 1402.11,
 1403.45,
 1441.47,
 1457.6,
 1438.56,
 1432.25,
 1449.68,
 1465.15,
 1455.14,
 1455.9,
 1445.57,
 1441.36,
 1401.53,
 1410.03,
 1404.09,
 1398.56,
 1360.16,
 1394.46,
 1409.28,
 1409.12,
 1424.97,
 1424.37,
 1424.24,
 1441.72,
 1411.71,
 1416.83,
 1387.12,
 1389.94,
 1402.05,
 1387.67,
 1388.26,
 1346.09,
 1352.17,
 1360.69,
 1353.43,
 1333.36,
 1348.05,
 1366.42,
 1379.19,
 1381.76,
 1409.17,
 1391.28,
 1355.62,
 1366.7,
 1401.69,
 1395.07,
 1383.62,
 1359.15,
 1392.14,
 1458.47,
 1464.47,
 1456.63,
 1493.87,
 1500.64,
 1527.35,
 1527.46,
 1523.86,
 1507.73,
 1508.52,
 1487.92,
 1498.58,
 1505.97,
 1494.73,
 1487.37,
 1501.34,
 1516.35,
 1504.46,
 1500.59,
 1467.17,
 1440.51,
 1356.56,
 1401.44,
 1441.61,
 1427.47,
 1434.54,
 1429.86,
 1477.44,
 1460.99,
 1464.92,
 1452.43,
 1468.25,
 1446.29,
 1415.1,
 1409.57,
 1432.63,
 1424.17,
 1412.14,
 1383.05,
 1407.81,
 1420.96,
 1452.36,
 1466.04,
 1447.8,
 1437.21,
 1406.95,
 1400.72,
 1373.86,
 1399.05,
 1381

## 復習一下一些清單的實用功能

## slicing 

取出一個清單裏部分的值

```python
tsmc_prices[0:3]
```
0 代表起始的索引值、3代表結束的索引值的後面一個值

最後就會得到 [1455.22, 1399.42, 1402.11]

我們可以從 list 第 20 個值開始，取出該值與前 19 個值(20筆收盤價)，
使用 slicing 就可以讓我們用一行做出這個效果：
```python
tsmc_prices[i-19:i]
```

---
## .append()

把資料加到清單的最後面：

```
sma = [1,2,3]
sma.append(4)
```
最後產生的 list 就會是：
```python
[1, 2, 3, 4]
```

## numpy.std(list)
算出清單内資料(通常是一個數列)的**標準差**

## numpy.mean(list)
算出清單内資料(通常是一個數列)的**平均值**

In [6]:
# 先引用 numpy 模組
import numpy as np

In [7]:
# 計算標準差
std_arr = []
for i in range(19, len(tsmc_prices)):
    period = tsmc_prices[i-19:i]
    stdev = np.std(period)
    std_arr.append(stdev)

std_arr

[28.025405619023676,
 28.114806983956914,
 27.744398840775258,
 27.48904130175789,
 27.020693475561973,
 26.77564647812131,
 25.66051266536404,
 25.76830714629109,
 25.811746880215434,
 25.01966511884762,
 23.766801702330397,
 22.640505681343004,
 20.37616598298621,
 19.17903132744923,
 17.838098960237744,
 22.112697784849765,
 24.589430954811455,
 25.9486943933644,
 27.641993315602853,
 29.91824454721071,
 31.510588397607474,
 31.587696384524243,
 31.229492027555764,
 29.868886803695357,
 28.96549815042879,
 27.39727497216242,
 23.952089760146542,
 22.576948028275844,
 21.30052405946288,
 21.635116396632714,
 21.437279228489384,
 20.585744442167446,
 20.793160459238926,
 28.28295057229343,
 33.652310404604734,
 36.76640805434817,
 43.179095498732295,
 48.09336906396181,
 52.80614986639256,
 56.41317215684457,
 59.19156221686609,
 60.34333189011945,
 60.90861514508408,
 61.121485516058506,
 60.62260329671527,
 57.08751650024901,
 52.960704682275626,
 50.74235535671174,
 47.687076299385

In [8]:
# 計算平均值
mean_arr = []
for i in range(19, len(tsmc_prices)):
    period = tsmc_prices[i-19:i]
    mean = np.mean(period)
    mean_arr.append(mean)

mean_arr

[1427.2236842105262,
 1424.0257894736842,
 1424.544736842105,
 1424.913684210526,
 1426.0463157894735,
 1425.1463157894736,
 1423.3905263157894,
 1423.5568421052633,
 1422.4757894736842,
 1420.7468421052633,
 1416.64,
 1413.2084210526316,
 1410.3742105263157,
 1407.326842105263,
 1404.5321052631577,
 1401.6142105263157,
 1398.5689473684208,
 1396.2847368421053,
 1393.9094736842103,
 1392.498947368421,
 1390.0563157894735,
 1387.8005263157895,
 1386.2252631578945,
 1383.9510526315785,
 1383.1510526315788,
 1381.4163157894734,
 1376.884736842105,
 1374.515789473684,
 1373.7189473684211,
 1374.1373684210528,
 1373.8047368421053,
 1371.546842105263,
 1371.782105263158,
 1375.4773684210527,
 1381.7078947368423,
 1387.2057894736845,
 1394.2152631578947,
 1401.9631578947367,
 1412.1731578947367,
 1421.6157894736841,
 1429.9021052631579,
 1436.6673684210527,
 1443.3389473684213,
 1447.4836842105265,
 1453.1310526315792,
 1461.044210526316,
 1467.7826315789473,
 1472.292105263158,
 1477.8852631

In [9]:
# 計算下限 (Lower Bound)
min_arr = []
for i in range(0, len(mean_arr)):
    minimun = mean_arr[i] - 2 * std_arr[i]
    min_arr.append(minimun)

min_arr

[1371.172872972479,
 1367.7961755057704,
 1369.0559391605545,
 1369.9356016070103,
 1372.0049288383495,
 1371.595022833231,
 1372.0695009850613,
 1372.020227812681,
 1370.8522957132534,
 1370.7075118675682,
 1369.1063965953392,
 1367.9274096899455,
 1369.6218785603432,
 1368.9687794503645,
 1368.8559073426823,
 1357.3888149566162,
 1349.390085458798,
 1344.3873480553766,
 1338.6254870530047,
 1332.6624582739996,
 1327.0351389942587,
 1324.625133546741,
 1323.7662791027828,
 1324.2132790241878,
 1325.2200563307213,
 1326.6217658451485,
 1328.9805573218118,
 1329.3618934171323,
 1331.1178992494954,
 1330.8671356277873,
 1330.9301783851265,
 1330.3753532209282,
 1330.19578434468,
 1318.9114672764658,
 1314.4032739276329,
 1313.6729733649881,
 1307.85707216043,
 1305.776419766813,
 1306.5608581619517,
 1308.789445159995,
 1311.5189808294258,
 1315.9807046408139,
 1321.521717078253,
 1325.2407131784094,
 1331.8858460381487,
 1346.869177525818,
 1361.861222214396,
 1370.8073945497345,
 1382.

In [10]:
# 計算上限 (Upper Bound)
max_arr = []
for i in range(0, len(mean_arr)):
    maximum = mean_arr[i] + 2 * std_arr[i]
    max_arr.append(maximum)

max_arr

[1483.2744954485736,
 1480.255403441598,
 1480.0335345236556,
 1479.8917668140418,
 1480.0877027405975,
 1478.6976087457163,
 1474.7115516465176,
 1475.0934563978456,
 1474.099283234115,
 1470.7861723429585,
 1464.173603404661,
 1458.4894324153177,
 1451.126542492288,
 1445.6849047601615,
 1440.2083031836332,
 1445.8396060960151,
 1447.7478092780436,
 1448.182125628834,
 1449.193460315416,
 1452.3354364628426,
 1453.0774925846883,
 1450.975919084838,
 1448.684247213006,
 1443.688826238969,
 1441.0820489324362,
 1436.2108657337983,
 1424.7889163623981,
 1419.6696855302357,
 1416.319995487347,
 1417.4076012143182,
 1416.679295299084,
 1412.718330989598,
 1413.3684261816359,
 1432.0432695656395,
 1449.0125155460516,
 1460.7386055823808,
 1480.5734541553593,
 1498.1498960226604,
 1517.7854576275217,
 1534.4421337873732,
 1548.28522969689,
 1557.3540322012916,
 1565.1561776585895,
 1569.7266552426436,
 1574.3762592250098,
 1575.219243526814,
 1573.7040409434985,
 1573.7768159765815,
 1573.2

## numpy.reshape()

改變一個陣列的維度

假設原本我們有一個 1 x 6 的陣列：

```python
[0, 1, 2, 3, 4, 5]
```

我們將該陣列轉成一個 numpy 的陣列

```python
np.array(ary)
```

array([0, 1, 2, 3, 4, 5])

再用 reshape 把它變成一個 3 x 2 的陣列

```python
np.array(ary).reshape(3,2)
```

最後的陣列就會變成：

```python
array([[0, 1],
       [2, 3],
       [4, 5]])
```

In [11]:
mean_arr = np.array(mean_arr).reshape(len(mean_arr), 1)
min_arr = np.array(min_arr).reshape(len(min_arr), 1)
max_arr = np.array(max_arr).reshape(len(max_arr), 1)
mean_arr

array([[1427.22368421],
       [1424.02578947],
       [1424.54473684],
       ...,
       [ 875.95      ],
       [ 878.75052632],
       [ 880.95210526]])

In [12]:
tsmc_sheet.range((21, 'C')).value = min_arr
tsmc_sheet.range((21, 'D')).value = mean_arr
tsmc_sheet.range((21, 'E')).value = max_arr

In [15]:
# 實作 Bollinger band 交易 (Naive 方法)

# 目前是否已做多或做空的訊號
hold_flag = False
# 買入股票的訊號
long_flag = False
# 做空股票的訊號
short_flag = False
# 總收益
balance = 0

ts = time.time()

for i in range(21, len(tsmc_prices)+1):
    # 截取每一天的價格、下限、平均、與上限
    price = tsmc_sheet.cells(i, "B").value
    min = tsmc_sheet.cells(i, "C").value
    mean = tsmc_sheet.cells(i, "D").value
    max = tsmc_sheet.cells(i, "E").value
    
    # 若目前沒有進行交易
    if hold_flag == False:
        # 價格低於下限
        if price < min:
            # 顯示做多
            tsmc_sheet.cells(i, "F").value = "Long"
            # 計算總收益
            balance = balance - (price * 1000)
            # 花費現金購買股票
            tsmc_sheet.cells(i, "G").value = balance
            # 開啓交易訊號
            long_flag = True
            hold_flag = True
        # 價格高於上限
        if price > max:
            # 顯示做空
            tsmc_sheet.cells(i, "F").value = "Short"
            # 做空，現金增加
            balance = balance + (price * 1000)
            tsmc_sheet.cells(i, "G").value = balance
            # 開啓交易訊號
            short_flag = True
            hold_flag = True
    # 若目前有進行交易
    elif hold_flag == True:
       # 若現在是做多，而且價格大於等於平均值，平倉
       if long_flag == True and price >= mean:
            tsmc_sheet.cells(i, "F").value = "Offset"
            balance = balance + price * 1000
            tsmc_sheet.cells(i, "G").value = balance
            hold_flag = False
            long_flag = False
       # 若現在是做空，而且價格小於等於平均值，平倉
       elif short_flag == True and price <= mean:
            tsmc_sheet.cells(i, "F").value = "Offset"
            balance = balance - price * 1000
            tsmc_sheet.cells(i, "G").value = balance
            hold_flag = False
            short_flag = False

# 將最後的總收益顯示在 J5
tsmc_sheet.range('J5').value = balance

td = time.time()

time_spent = td - ts
time_spent

40.60543489456177

In [14]:
# 實作 Bollinger band 交易 (Array 版)

# 截取出從第 20 天開始的股價
tsmc_prices_trading = tsmc_prices[19:]
# 目前是否已做多或做空的訊號
hold_flag = False
# 買入股票的訊號
long_flag = False
# 做空股票的訊號
short_flag = False
# 記錄總收益
balance = 0

trading_results = []

ts = time.time()

for i in range(0, len(tsmc_prices_trading)):
    # 截取每一天的價格、下限、平均、與上限
    price = tsmc_prices_trading[i]
    min = min_arr[i][0]
    mean = mean_arr[i][0]
    max = max_arr[i][0]
    result = []
    # print("price: {}, max:{}, min:{}, mean:{}".format(price, max, min, mean))

    # 若目前沒有進行交易
    if hold_flag == False:
        # 價格低於下限
        if price < min:
            # 顯示做多
            trading_results.append("Long")
            # 計算總收益
            balance = balance - (price * 1000)
            # 花費現金購買股票
            trading_results.append(balance)
            # 開啓交易訊號
            long_flag = True
            hold_flag = True
        # 價格高於上限
        elif price > max:
            # 顯示做空
            trading_results.append("Short")
            # 做空，現金增加
            balance = balance + (price * 1000)
            trading_results.append(balance)
            # 開啓交易訊號
            short_flag = True
            hold_flag = True
        else:
            trading_results.append("")
            trading_results.append(balance)
    # 若目前有進行交易
    elif hold_flag == True:
       # 若現在是做多，而且價格大於等於平均值，平倉
       if long_flag == True and price >= mean:
            trading_results.append("Offset")
            # 做多在平倉時，現金增加
            balance = balance + price * 1000
            trading_results.append(balance)
            hold_flag = False
            long_flag = False
       # 若現在是做空，而且價格小於等於平均值，平倉
       elif short_flag == True and price <= mean:
            trading_results.append("Offset")
            # 做空在平倉時，現金減少
            balance = balance - price * 1000
            trading_results.append(balance)
            hold_flag = False
            short_flag = False
       else:
            trading_results.append("")
            trading_results.append(balance)

# 把 ["long", 123, 0, 0, "short", 456, ...] 的清單轉成一個 n/2 x 2 的 numpy array
trading_results = np.array(trading_results).reshape(int(len(trading_results)/2), 2)
# 最後再一次性的寫入 Excel，只需要指定範圍的起點 (左上角) 即可
tsmc_sheet.range('F21').value = trading_results

td = time.time()

# 將最後的總收益顯示在 J5
tsmc_sheet.range('J5').value = balance
# 算出花費的時間
time_spent_arr = td - ts
time_spent_arr

0.1574265956878662

In [17]:
# 可以看出，使用陣列運算在執行效能的差距是非常巨大的，而且處理的資料量越大，這個差異就越明顯
print("Array 版時間差距：{}".format(time_spent / time_spent_arr))

Array 版時間差距：146.41650998668328


# 結論
- 當你處理的資料**提升了一個量級，若不用聰明一點的方法寫程式，執行出來的效能(時間)就會有一個巨大的差距**
- **減少 Excel 儲存格的讀取次數，是最簡單的省時方法**
- 所以盡量將需要計算的資料暫時存入陣列，避免讓程式直接讀取儲存格 (用時間換空間)